Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [15]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [16]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [17]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

train_subset = 10000

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [18]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.



In [8]:
# Logistic Regression model with L2 Loss

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta*tf.nn.l2_loss(weights) + beta*tf.nn.l2_loss(biases)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.662514
Minibatch accuracy: 13.3%
Validation accuracy: 17.7%
Minibatch loss at step 500: 2.658278
Minibatch accuracy: 78.9%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.487462
Minibatch accuracy: 85.2%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 1.111572
Minibatch accuracy: 79.7%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 0.950928
Minibatch accuracy: 80.5%
Validation accuracy: 80.4%
Minibatch loss at step 2500: 0.904721
Minibatch accuracy: 75.8%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 0.633830
Minibatch accuracy: 85.9%
Validation accuracy: 81.3%
Test accuracy: 88.2%


In [21]:
# One layer hidden nueral network with relu unit and regularization

batch_size = 128
hidden1_units = 1024
beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Hidden layer 1 Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))
    
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  
 
  # Training computation. 
   
  weights2 = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  logits = tf.matmul(hidden1, weights2) + biases2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(biases1) + beta*tf.nn.l2_loss(weights2) + beta*tf.nn.l2_loss(biases2) 
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 676.070435
Minibatch accuracy: 7.0%
Validation accuracy: 25.4%
Minibatch loss at step 500: 197.198990
Minibatch accuracy: 79.7%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 114.789764
Minibatch accuracy: 82.8%
Validation accuracy: 82.1%
Minibatch loss at step 1500: 69.016960
Minibatch accuracy: 83.6%
Validation accuracy: 82.4%
Minibatch loss at step 2000: 41.745129
Minibatch accuracy: 82.0%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.646980
Minibatch accuracy: 82.0%
Validation accuracy: 86.0%
Minibatch loss at step 3000: 15.538237
Minibatch accuracy: 89.8%
Validation accuracy: 86.5%
Test accuracy: 92.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [25]:
# One layer hidden nueral network with relu unit, regularization and dropout

batch_size = 128
hidden1_units = 1024
hidden2_units = 1024
beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Hidden layer 1 Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))
    
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

  #add dropout on hidden layer
  #we pick up the probabylity of switching off the activation
  #and perform the switch off of the activations
  #keep_prob = tf.placeholder("float")
  #hidden1_drop = tf.nn.dropout(hidden1, keep_prob)  
    
  #add dropout on hidden layer
  #we pick up the probabylity of switching off the activation
  #and perform the switch off of the activations
  keep_prob = tf.placeholder("float")
  hidden1_drop = tf.nn.dropout(hidden1, keep_prob)  
  
 
  # Output layer. 
   
  weights2 = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  logits = tf.matmul(hidden1, weights2) + biases2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(biases1) + beta*tf.nn.l2_loss(weights2) + beta*tf.nn.l2_loss(biases2) 
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 701.898499
Minibatch accuracy: 6.2%
Validation accuracy: 24.5%
Minibatch loss at step 500: 199.569901
Minibatch accuracy: 80.5%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 114.602394
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 69.189369
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 41.772564
Minibatch accuracy: 85.2%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 25.351263
Minibatch accuracy: 87.5%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 15.515548
Minibatch accuracy: 89.8%
Validation accuracy: 86.7%
Test accuracy: 92.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [31]:
# Multi layer hidden nueral network with relu unit, regularization and dropout

batch_size = 128
hidden1_units = 1024
hidden2_units = 1024
beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1_units], stddev=0.1))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))
    
  weights2 = tf.Variable(tf.truncated_normal([hidden1_units, hidden2_units], stddev=0.1))
  biases2 = tf.Variable(tf.zeros([hidden2_units]))
    
  weights3 = tf.Variable(tf.truncated_normal([hidden2_units, num_labels], stddev=0.1))
  biases3 = tf.Variable(tf.zeros([num_labels]))
    
  
  # Model.
  def model(data):
    hidden1 = tf.nn.relu(tf.matmul(data, weights1) + biases1)
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
    return tf.matmul(hidden2, weights3) + biases3
    
  
  logits = model(tf_train_dataset)
    
  vars   = tf.trainable_variables() 
  lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in vars])*beta
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + lossL2 
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 14.039816
Minibatch accuracy: 10.2%
Validation accuracy: 17.2%
Minibatch loss at step 500: 6.028149
Minibatch accuracy: 83.6%
Validation accuracy: 85.4%
Minibatch loss at step 1000: 3.709044
Minibatch accuracy: 89.8%
Validation accuracy: 86.2%
Minibatch loss at step 1500: 2.525301
Minibatch accuracy: 85.2%
Validation accuracy: 87.0%
Minibatch loss at step 2000: 1.679380
Minibatch accuracy: 89.8%
Validation accuracy: 88.0%
Minibatch loss at step 2500: 1.208229
Minibatch accuracy: 86.7%
Validation accuracy: 88.0%
Minibatch loss at step 3000: 0.817352
Minibatch accuracy: 91.4%
Validation accuracy: 88.2%
Test accuracy: 93.4%
